In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from get_data.data_align_and_save.process_tools import *
from concurrent import futures
import sys
sys.argv = ['']
from config import get_config
config = get_config()

In [2]:
from get_data.data_align_and_save.process_tools import *

In [3]:
f = pd.read_parquet('/home/USB_DRIVE1/Chenx_datawarehouse/5min_features_ori/volume_5min.parquet')

In [4]:
inner, outer = split_by_datetime_index(f)

In [5]:
market = pd.read_parquet('/home/hongkou/chenx/data_warehouse/marketcap.parquet')

In [6]:
market_inner, market_outer = split_by_datetime_index(market)

In [7]:
def minute_align_fix_with_market(factor_value:pd.DataFrame, marketcap:pd.DataFrame):
    '''
        因子值与marketcap对齐并填均(裁剪)值然后mask.
        Params:
            factor_value: 待对齐的因子值
            marketcap: 待对齐的marketcap
        Returns
        ---
        factor_value
            对齐后的因子值
    '''
    common_cols = factor_value.columns.union(marketcap.columns)
    factor_value = factor_value.reindex(columns=common_cols)
    marketcap = marketcap.reindex(columns=common_cols)
    factor_value = factor_value.apply(adjust_row, axis=1)
    marketcap.index = pd.to_datetime(marketcap.index)
    upsampled = marketcap.reindex(inner.index.normalize())
    upsampled.index = inner.index
    factor_value = factor_value.where(upsampled>0)

    return factor_value

In [8]:
inner_aligned = minute_align_fix_with_market(inner, market_inner)

In [9]:
def standard(df):
    return (df.sub(df.mean(axis=1), axis=0)).div(df.std(axis=1), axis=0)

In [10]:
inner_aligned_std = standard(inner_aligned)

In [11]:
from get_data.CY312.CY312_rollingtrain_dataloader import get_CY312_rollingtrain_CrossSectionLoader
CrossSection_trainloader, CrossSection_testloader = get_CY312_rollingtrain_CrossSectionLoader(batchsize="all", shuffle_time=False, time_period = config.time_period)

/home/hongkou/anaconda3/envs/timeseries/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/hongkou/anaconda3/envs/timeseries/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


total data shape: (3873, 969, 313)


/home/hongkou/anaconda3/envs/timeseries/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/hongkou/anaconda3/envs/timeseries/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


total data shape: (3873, 243, 313)


In [12]:
def lgb_train_and_test(trainloader, testloader):
    '''
    :return: 两个arr
    '''
    train_set = next(iter(trainloader))
    train_data = train_set.reshape(-1, train_set.shape[-1])
    mask = ~torch.isnan(train_data[:, -1])
    train_data = train_data[mask]

    # train_nan_mask = torch.isnan(train_data)
    # train_nan_counts = train_nan_mask.sum(dim=1)
    # train_indices = torch.where(train_nan_counts <= 8)[0]
    # train_data = train_data[train_indices]

    train_data = np.array(train_data)
    train_data_x = train_data[:, :-1]
    train_data_y = train_data[:, -1]

    model = lgb.LGBMRegressor(learning_rate=0.07, n_estimators=350)

    model.fit(train_data_x, train_data_y)
    model.booster_.save_model(os.path.join(config.exp_path, 'models/lgbm.txt'))

    test_pred, test_true = tree_test(testloader, model)     # (292, 3873)

    return test_pred, test_true

In [13]:
lgb_pred_arr, lgb_ture = lgb_train_and_test(CrossSection_trainloader, CrossSection_testloader)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.466646 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 79560
[LightGBM] [Info] Number of data points in the train set: 2949429, number of used features: 312
[LightGBM] [Info] Start training from score 0.000000

lgbm test ic: 0.07990158444568084
